In [39]:
import pandas as pd
from unicodedata import normalize
from unidecode import unidecode
import re
 
df = pd.read_csv('raw-address-input.csv')
# Normalizing address
df['Norm Address'] = df.apply(lambda x: normalize('NFC', str(x['Complete Address']).lower()), axis=1)
df['Province'] = None # Initialize Province column
df['District'] = None # Initialize Province column
df.head()

,Complete Address,Norm Address,Province,District
0,Căn hộ 3020; Toà S103; Kđt Vinhomes Smart city...,căn hộ 3020; toà s103; kđt vinhomes smart city...,None,None
1,1338 Đường 781 Bàu Năng Dương Minh Châu Tây Ni...,1338 đường 781 bàu năng dương minh châu tây ni...,None,None
2,24/7 Xô Viết Nghệ Tĩnh; Phường 19; Bình Thạnh ...,24/7 xô viết nghệ tĩnh; phường 19; bình thạnh ...,None,None
3,509; Nhạc Sơn; Phường cốc Lếu; Thành phố Lào C...,509; nhạc sơn; phường cốc lếu; thành phố lào c...,None,None
4,124/ 35/3 Xô Viết Nghệ Tĩnh; Phường 21; Bình T...,124/ 35/3 xô viết nghệ tĩnh; phường 21; bình t...,None,None


In [2]:
provDf = pd.read_csv('prov-optimized.csv')
# Add diacritic-less province column
provDf['Province Decoded'] = provDf.apply(lambda x: unidecode(x['Province']), axis=1)
provDf.head()

,Province,Province RegEx,Province Code,Province Decoded
0,Hà Nội,HN,1,Ha Noi
1,Hà Giang,NaN,2,Ha Giang
2,Cao Bằng,NaN,4,Cao Bang
3,Bắc Kạn,NaN,6,Bac Kan
4,Tuyên Quang,NaN,8,Tuyen Quang


In [3]:
distDf = pd.read_csv('dist-optimized.csv')
# Add diacritic-less district column
distDf['District Decoded'] = distDf.apply(lambda x: unidecode(x['District']), axis=1)
distDf.head()

,Province,District Code,District,District RegEx,District Decoded
0,Hà Nội,1,Ba Đình,NaN,Ba Dinh
1,Hà Nội,2,Hoàn Kiếm,NaN,Hoan Kiem
2,Hà Nội,3,Tây Hồ,NaN,Tay Ho
3,Hà Nội,4,Long Biên,NaN,Long Bien
4,Hà Nội,5,Cầu Giấy,NaN,Cau Giay


In [4]:
wardDf = pd.read_csv('ward-optimized.csv')
# Add diacritic-less ward column
wardDf['Ward Decoded'] = wardDf.apply(lambda x: unidecode(x['Ward']), axis=1)
wardDf.head()

,Province,District,Ward Code,Ward,Ward Decoded
0,Hà Nội,Ba Đình,1.0,Phúc Xá,Phuc Xa
1,Hà Nội,Ba Đình,4.0,Trúc Bạch,Truc Bach
2,Hà Nội,Ba Đình,6.0,Vĩnh Phúc,Vinh Phuc
3,Hà Nội,Ba Đình,7.0,Cống Vị,Cong Vi
4,Hà Nội,Ba Đình,8.0,Liễu Giai,Lieu Giai


In [5]:
df[df.Province.isna()]

,Complete Address,Norm Address,Province
0,Căn hộ 3020; Toà S103; Kđt Vinhomes Smart city...,căn hộ 3020; toà s103; kđt vinhomes smart city...,None
1,1338 Đường 781 Bàu Năng Dương Minh Châu Tây Ni...,1338 đường 781 bàu năng dương minh châu tây ni...,None
2,24/7 Xô Viết Nghệ Tĩnh; Phường 19; Bình Thạnh ...,24/7 xô viết nghệ tĩnh; phường 19; bình thạnh ...,None
3,509; Nhạc Sơn; Phường cốc Lếu; Thành phố Lào C...,509; nhạc sơn; phường cốc lếu; thành phố lào c...,None
4,124/ 35/3 Xô Viết Nghệ Tĩnh; Phường 21; Bình T...,124/ 35/3 xô viết nghệ tĩnh; phường 21; bình t...,None
...,...,...,...
995,Nhà số 3; ngách 36; ngõ 77 Xuân La; Tây Hồ Phư...,nhà số 3; ngách 36; ngõ 77 xuân la; tây hồ phư...,None
996,"Số nhà 11, ngách 49, ngõ 14, Mễ Trì Hạ,...","số nhà 11, ngách 49, ngõ 14, mễ trì hạ, từ liê...",None
997,Cc ssr nguyễn hữu thọ nhà bè Saigon south resi...,cc ssr nguyễn hữu thọ nhà bè saigon south resi...,None
998,35/12 Bế Văn Cấm; Tân Kiểng; Quận 7; Hồ Chí Mi...,35/12 bế văn cấm; tân kiểng; quận 7; hồ chí mi...,None


In [30]:
# Function for parsing districts first
def parseDist(df):
  tempDf = df[df.Province.isna()]
  for i in tempDf.index:
    dist = None
    for m in distDf.index:
      di = distDf["District"][m]
      if di.isnumeric():
        di = "Q.{0,6}" + di
      tempDist = re.search(di.lower(), df["Norm Address"][i])
      if tempDist == None:
        continue
      else:
        if dist == None or tempDist.span()[0] < dist.span()[0]:
          dist = tempDist
          df.loc[i, "District"] = distDf["District"][m]
          df.loc[i, "Province"] = distDf["Province"][m]
    
    for n in distDf.index:
      di = distDf["District RegEx"][n]
      if isinstance(di, float):
              continue
      if di.isnumeric() or di == "(2|9)":
        di = "Q.{0,6}" + di
      tempDist = re.search(di.lower(), df["Norm Address"][i])
      if tempDist == None:
        continue
      else:
        if dist == None or tempDist.span()[0] < dist.span()[0]:
          dist = tempDist
          df.loc[i, "District"] = distDf["District"][n]
          df.loc[i, "Province"] = distDf["Province"][n]
  
  for k in tempDf.index:
      ward = None
      for w in wardDf[wardDf["District"] == df["District"][k]]["Ward"]:
          if isinstance(w, float):
              continue
          wa = w
          if wa.isnumeric():
              wa = "P.{0,8}" + wa
          tempWard = re.search(wa.lower(), df["Norm Address"][k])
          if tempWard == None:
              continue
          else:
              if ward == None or tempWard.span()[0] < ward.span()[0]:
                  ward = tempWard
                  df.loc[k, "Ward"] = w
      
      

In [21]:
# Function for top-down parsing
def parse(df, provType, distType, wardType):
    tempDf = df[df.Province.isna() | df.District.isna()]
    for i in tempDf.index:
        city = None
        for m in provDf[provType].index:
            c = provDf[provType][m]
            if isinstance(c, float):
                continue
            tempCity = re.search(c.lower(), df["Norm Address"][i])
            if tempCity == None:
                continue
            else:
                if city == None or tempCity.span()[0] < city.span()[0]:
                    city = tempCity
                    df.loc[i, "Province"] = provDf.Province[m]

    for j in tempDf.index:
        dist = None
        tempDistDf = distDf[distDf["Province"] == df["Province"][j]]
        for n in tempDistDf.index:
            d = tempDistDf[distType][n]
            if isinstance(d, float):
                continue
            di = d
            if di.isnumeric() or di == "(2|9)": # "Thu Duc city = District 2 & 9"
                di = "Q.{0,6}" + di
            tempDist = re.search(di.lower(), df["Norm Address"][j])
            if tempDist == None:
                continue
            else:
                if dist == None or tempDist.span()[0] < dist.span()[0]:
                    dist = tempDist
                    df.loc[j, "District"] = tempDistDf.District[n]

    for k in tempDf.index:
        ward = None
        for w in wardDf[wardDf["District"] == df["District"][k]][wardType]:
            if isinstance(w, float):
                continue
            wa = w
            if wa.isnumeric():
                wa = "P.{0,8}" + wa
            tempWard = re.search(wa.lower(), df["Norm Address"][k])
            if tempWard == None:
                continue
            else:
                if ward == None or tempWard.span()[0] < ward.span()[0]:
                    ward = tempWard
                    df.loc[k, "Ward"] = w
    

In [40]:
parse(df, "Province", "District", "Ward") # Classic parsing
parse(df, "Province Decoded", "District Decoded", "Ward Decoded") # Parsing without diacritics
parse(df, "Province RegEx", "District", "Ward") # Parsing with abbreviation names
parse(df, "Province", "District RegEx", "Ward") # Parsing with abbreviation names
parseDist(df) # Parsing district and refer to province

In [43]:
# Test code
df.head(50)

,Complete Address,Norm Address,Province,District,Ward
0,Căn hộ 3020; Toà S103; Kđt Vinhomes Smart city...,căn hộ 3020; toà s103; kđt vinhomes smart city...,Hà Nội,Nam Từ Liêm,Tây Mỗ
1,1338 Đường 781 Bàu Năng Dương Minh Châu Tây Ni...,1338 đường 781 bàu năng dương minh châu tây ni...,Tây Ninh,Dương Minh Châu,Bàu Năng
2,24/7 Xô Viết Nghệ Tĩnh; Phường 19; Bình Thạnh ...,24/7 xô viết nghệ tĩnh; phường 19; bình thạnh ...,Hồ Chí Minh,Bình Thạnh,19
3,509; Nhạc Sơn; Phường cốc Lếu; Thành phố Lào C...,509; nhạc sơn; phường cốc lếu; thành phố lào c...,Lào Cai,Lào Cai,Cốc Lếu
4,124/ 35/3 Xô Viết Nghệ Tĩnh; Phường 21; Bình T...,124/ 35/3 xô viết nghệ tĩnh; phường 21; bình t...,Hồ Chí Minh,Bình Thạnh,21
5,chung cư 62 bà hom; Phường 13; Quận 6; Hồ Chí ...,chung cư 62 bà hom; phường 13; quận 6; hồ chí ...,Hồ Chí Minh,6,13
6,Hồng Hà; Phường 2; Quận Tân Bình; HCM,hồng hà; phường 2; quận tân bình; hcm,Hồ Chí Minh,Tân Bình,2
7,11 LÔ J4 KP4 ĐƯỜNG DD12 Phường Tân Hưng Thuận ...,11 lô j4 kp4 đường dd12 phường tân hưng thuận ...,Hồ Chí Minh,12,Tân Hưng Thuận
8,Chung cư Safira Khang Điền; Phường Phú Hữu; Qu...,chung cư safira khang điền; phường phú hữu; qu...,Hồ Chí Minh,Thủ Đức,Phú Hữu
9,Block B; Thảo Điền Pearl; 12 Quốc Hương; phườn...,block b; thảo điền pearl; 12 quốc hương; phườn...,Hồ Chí Minh,Thủ Đức,Thảo Điền


In [47]:
# Save the CSV file
df.to_csv("address-output.csv", encoding='utf-8-sig', index = False)